# Goals for AIGAME - RPS (Rock, Paper, Scissors) Part 4

### Using the DataFrame to represent RPS games
In the last tutorial, we saw how to represent a RPS game using a Python class which we wrote. But we will not use that class, instead we will use a class from a third party library.
The class we will use is the DataFrame class from the Pandas package.

Why?

1.  By choosing an existing, proven, third-party class, we can worry more about our AI logic and less about writing Python classes
1.  The DataFrame class is extremely flexible and powerful
1.  Most importantly, many existing Machine Learning packages work with DataFrames. So by starting with DataFrames, we will more easily be able to implement Machine Learning algorithms


### Saving/loading a list of games
In the previous tutorial, we saw how to represent a collection of RPS class instances. When we switch to using Pandas DataFrames, we will find that not only will it be able to represent a single game, it will be able to represent a collection of games. **Futher**, it will be able to save/load those as well.

The above is an important part of the outline for our AI game:
1.  Load the history of previous games
1.  Analyze that history and formulate a strategy
1.  Play a game and store results
1.  Re-examine history and make changes.
1.  When done playing, store all game results for next time



# Build Experience
To see how we will do the above, let's introduce ourselves to the DataFrame class and play around with it a bit.

### The DataFrame is a class exposed by the Pandas library which can represent a collection of records.
Think of a Spreadsheet:
1.  Each column represents a different attribute (e.g. Name, Address)
1.  Each row represents a separate record, but all records have the same attributes/columns

|NAME| ZIP |GENDER|
|----|-----|------|
|Bob |12033|  Male|
|Rita|34444|Female|
|Pete|11221|  Male|

We can do the same using a DataFrame as follows:


In [1]:
import pandas as pd
columnNames = ['NAME','ZIP','GENDER']
data = [['Bob',12033,'Male'], ['Rita',34444,'Female'],['Pete',11221,'Male']]
dfTest = pd.DataFrame(columns=columnNames, data=data)
print(dfTest)

   NAME    ZIP  GENDER
0   Bob  12033    Male
1  Rita  34444  Female
2  Pete  11221    Male


In [2]:
# In the above example, each row represents a person. 
# Let's create a DataFrame where each row represents a game of RPS
columnNames = ['AI Response','Human Response', 'Outcome', 'Outcome Text']
print(columnNames)

['AI Response', 'Human Response', 'Outcome', 'Outcome Text']


In [3]:
###################
# Rock, Paper or Scissors
ROCK=1
PAPER=2
SCISSORS=3
#AI Win, Human Win or DRAW
DRAW=0
AI_WIN=1
HUMAN_WIN=-1
#####################################

#Now let's create a matching list of data for those columns
game1 = [ROCK, PAPER, HUMAN_WIN, 'Human Wins!']
game2 = [ROCK, ROCK, DRAW, 'DRAW!']
game3 = [SCISSORS, ROCK, HUMAN_WIN, 'Human Wins!']
all_games = [game1,game2,game3]  #look familiar??
print(all_games)

[[1, 2, -1, 'Human Wins!'], [1, 1, 0, 'DRAW!'], [3, 1, -1, 'Human Wins!']]


In [4]:
#First, we have to import the pandas package (by standard convention, it is abbreviated as pd)
import pandas as pd 

dsGames = pd.DataFrame(columns=columnNames, data=all_games)
print(dsGames)


   AI Response  Human Response  Outcome Outcome Text
0            1               2       -1  Human Wins!
1            1               1        0        DRAW!
2            3               1       -1  Human Wins!


### Saving and Loading DataFrame
All of our hard work is about to pay off. You may have been worried when you saw the DataFrame and thought that everything is getting complicated. But the Pandas DataFrame is extremely powerful and will reward us for our efforts. Let's write some functions for saving and loading. Don't worry, they will not be long!

In [5]:
def SaveGameResults(fileSavePath, resultsDataFrame):
    '''This function takes two arguments:
    1) fileSavePath - the full path to the file which will hold the results, e.g. C:\RPS_AI\Results.csv (it will be a csv)
    2) resultsDataFrame - a DataFrame which holds all the results
    '''
    with pd.ExcelWriter(fileSavePath) as writer:
        resultsDataFrame.to_excel(writer, sheet_name="RPS Results", index=False)
        writer.save()
def LoadGameResults(fileLoadPath):
    '''This function takes one argument (the path of the saved results) and returns a DataFrame'''
    retvalue = pd.read_excel(fileLoadPath, sheet_name="RPS Results")
    return retvalue

Too good to be true? Let's try it out.

In [6]:
#TODO replace the value of RPS_RESULT_PATH so that it points to a folder on your computer
RPS_RESULTS_PATH = 'C:/Users/LEN320/Google Drive/DEV/Python/rps_results.xls'
print('Saving our DataFrame to ', RPS_RESULTS_PATH)
SaveGameResults(RPS_RESULTS_PATH, dsGames)

Saving our DataFrame to  C:/Users/LEN320/Google Drive/DEV/Python/rps_results.xls


You can go to that folder and verify the file was created. But we can also use a python funcion to see that the file exists

In [7]:
import os
if not os.path.exists(RPS_RESULTS_PATH):
    print('Something went wrong')
else:
    print('File was created')

File was created


Now let's load the information we just saved back into a new DataFrame

In [8]:
dsTest = LoadGameResults(RPS_RESULTS_PATH)
dsTest.head()

,AI Response,Human Response,Outcome,Outcome Text
0,1,2,-1,Human Wins!
1,1,1,0,DRAW!
2,3,1,-1,Human Wins!


# Execute - Loading, Adding To and Saving RPS Results

Let's use the code we wrote here and previous chapters to write a full sample game using random selections for both human and AI

In [9]:
import pandas as pd 
import random
import os
################## SOME OLD FUNCTIONS TO HELP US WRITE THIS #############################
###################
# Rock, Paper or Scissors
ROCK=1
PAPER=2
SCISSORS=3
#AI Win, Human Win or DRAW
DRAW=0
AI_WIN=1
HUMAN_WIN=-1
#####################################
def GetGameOutcome(aiChoice, humanChoice):
    outcome = None
    if aiChoice==humanChoice:
        outcome = 0 #Both chose the same thing, this is a draw
    elif aiChoice==ROCK and humanChoice==SCISSORS:
        outcome = AI_WIN #AI rock breaks scissors - AI win
    elif aiChoice==PAPER and humanChoice==ROCK:
        outcome = AI_WIN #AI paper covers rock - AI win
    elif aiChoice==SCISSORS and humanChoice == PAPER:
        outcome = AI_WIN #AI scissors cuts paper - AI win
    else:
        outcome = HUMAN_WIN #if it was not a draw and it is not an AI win, then it must be an AI loss
    return outcome

def GetOutcomeText(outcome):
    if outcome==DRAW:
        return "Draw!"
    elif outcome==AI_WIN:
        return "AI Wins!"
    elif outcome==HUMAN_WIN:
        return "Human Wins!!"
    else:
        return '??'  #this is an error
    
import random # package to help us generate random RPS responses
def getRandomRPSChoice():
    '''
    Return a random choice 1 (ROCK), 2(PAPER) or 3(SCISSORS)
    '''
    ichoice =  random.randrange(1,4)  #Generate a random number that is >= 1 and <4 (1,2 or 3)
    return ichoice

def getPlayerChoice():
    '''
    Prompt the human player to choose Rock, Paper or Scissors by entering in a number
    '''
    ok = False
    while not ok:
        response = input('1=Rock 2=Paper 3=Scissors')
        if (response=='1') or (response=='2') or (response=='3'):
            break
    return int(response)

def SaveGameResults(fileSavePath, resultsDataFrame):
    '''This function takes two arguments:
    1) fileSavePath - the full path to the file which will hold the results, e.g. C:\RPS_AI\Results.csv (it will be a csv)
    2) resultsDataFrame - a DataFrame which holds all the results
    '''
    with pd.ExcelWriter(fileSavePath) as writer:
        resultsDataFrame.to_excel(writer, sheet_name="RPS Results", index=False)
        writer.save()
def LoadGameResults(fileLoadPath):
    '''This function takes one argument (the path of the saved results) and returns a DataFrame'''
    if not os.path.exists(RPS_RESULTS_PATH):
        #create an empty DataFrame
        columnNames = ['AI Response','Human Response', 'Outcome', 'Outcome Text']
        retvalue = pd.DataFrame(columns=columnNames)
    else:
        #load from file
        retvalue = pd.read_excel(fileLoadPath, sheet_name="RPS Results")
    return retvalue

###################################################################################

In [10]:
#################### SOME NEW FUNCTIONS DRAWN FROM THE ABOVE ###########################
def AnalyzeGameHistory(ds):
    ''' TODO in a future tutorial: apply machine learnng techniques
    '''
def AddGameToResults(aiResponse, humanResponse, outcome, outcomeText, dsCurrentResults):
    #print(dsCurrentResults.index)
    retvalue = dsCurrentResults.append( {"AI Response":aiResponse,"Human Response":humanResponse,"Outcome":outcome,"Outcome Text":outcomeText}, ignore_index=True )
    #print(retvalue.head(20))
    return retvalue
############################################################################################    

In [11]:
######################### GAME (First Draft) ############################################
RPS_RESULTS_PATH = 'C:/Users/LEN320/Google Drive/DEV/Python/rps_results.xls'
NUM_GAMES_TO_PLAY = 30

#Load previous results (if the file exists)
dsGameHistory = LoadGameResults(RPS_RESULTS_PATH)

#Let AI study those results so that it can play well
AnalyzeGameHistory(dsGameHistory)

#Play some games
for igame in range(NUM_GAMES_TO_PLAY):
    human_choice = getPlayerChoice() #TODO - in future, we will ask the human to enter their choice
    ai_choice = getRandomRPSChoice() #TODO - in future, use machine learning algorithms to choose
    outcome = GetGameOutcome(aiChoice=ai_choice, humanChoice=human_choice)
    outcomeText = GetOutcomeText(outcome)
    print(outcomeText)
    dsGameHistory = AddGameToResults(ai_choice, human_choice, outcome, outcomeText, dsGameHistory)

#Save full history
SaveGameResults(RPS_RESULTS_PATH, dsGameHistory)


##########################################################################################333333

1=Rock 2=Paper 3=Scissors1
Human Wins!!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors1
Human Wins!!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Paper 3=Scissors2
Draw!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Paper 3=Scissors2
AI Wins!
1=Rock 2=Paper 3=Scissors3
Draw!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors2
AI Wins!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors3
Draw!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Paper 3=Scissors3
Human Wins!!
1=Rock 2=Paper 3=Scissors3
AI Wins!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Paper 3=Scissors2
AI Wins!
1=Rock 2=Paper 3=Scissors2
Draw!
1=Rock 2=Paper 3=Scissors1
Human Wins!!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors1
Draw!
1=Rock 2=Paper 3=Scissors2
AI Wins!
1=Rock 2=Paper 3=Scissors3
Draw!
1=Rock 2=Paper 3=Scissors1
AI Wins!
1=Rock 2=Pap

In [12]:
print(dsGameHistory)

    AI Response  Human Response  Outcome  Outcome Text
0             1               2       -1   Human Wins!
1             1               1        0         DRAW!
2             3               1       -1   Human Wins!
3             3               1       -1  Human Wins!!
4             1               1        0         Draw!
5             1               1        0         Draw!
6             3               1       -1  Human Wins!!
7             1               1        0         Draw!
8             2               1        1      AI Wins!
9             2               2        0         Draw!
10            2               1        1      AI Wins!
11            3               2        1      AI Wins!
12            3               3        0         Draw!
13            1               1        0         Draw!
14            3               2        1      AI Wins!
15            1               1        0         Draw!
16            3               3        0         Draw!
17        

### Wrapup
We put together alot of code but hopefully you'll see that *most* of it came from previous tutorials. Still it's alot. But if you look at the above you'll see that we have done almost everything.

In the next tutorial we will tackle the part about using history to determine game strategy: the meat of machine learning.
